**Batter Spray Charts**

This notebook generates a spray chart showing away and home team batting results for a specified game.

The field dimensions are notional becuase every ball park is different and not symetrical.  The assumptions are that both the left and right foul poles are about 320' from home plate and the straight away wall in centerfield is 400'.  A smooth arc connects the 400' centerfield point to each foul pole.  This is not the case in most ball parks.  Thus, some home runs may not appear to clear the park, where other hits may seem to be over the fence.  

The infield arc is also notional but provides a general demarkation between the infield and outfield.

The pie wedge divisions are rougly 1/3 of the x-axis, which distorts the eveness of thier area when the outfield corners are considered.  Again, this is just an estimation to provide context.

The gist of this chart is to show the locations of hits (which is accurate) for each team so general trends and observations can be deduced when combined with other facts, such as pitcher-batter match ups, shifts, etc.

In [ ]:
# load some modules and setup constants

import requests
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import path 
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np
import sys
%matplotlib inline

# MLB API
API_BASE_URL = "http://statsapi.mlb.com/api"
API_PLAYBYPLAY_URL = API_BASE_URL + "/v1/game/{}/playByPlay?fields={}"
API_SCHEDULE_URL = API_BASE_URL + "/v1/schedule?sportId=1&date={}&fields={}"
API_TEAMS_URL = API_BASE_URL + "/v1/teams?sportId=1&activeStatus=ACTIVE&fields={}"

# Team name dictionary
TEAM_NAMES_BY_ABBREV = {}

# chart constants
MAX_Y = 500
MIN_Y = 0
Y_OFFSET = 500
MAX_X = 240 
X_ONE_THIRD = MAX_X / 3
Y_ONE_THIRD = MAX_Y / 3
MIN_X = 0
MID_X = (MAX_X - MIN_X) / 2

# setup polygons to represent sectors (pie wedges) of field (thirds)
POLY_LEFT_PTS = [[MID_X,MIN_Y], [MIN_X,Y_ONE_THIRD * 2], [MIN_X,MAX_Y], [X_ONE_THIRD,MAX_Y], [MID_X,MIN_X]]
POLY_MIDDLE_PTS = [[MID_X,MIN_X], [X_ONE_THIRD,MAX_Y], [X_ONE_THIRD * 2,MAX_Y], [MID_X,MIN_X]]
POLY_RIGHT_PTS = [[MID_X,MIN_X], [X_ONE_THIRD * 2,MAX_Y], [MAX_X,MAX_Y], [MAX_X,Y_ONE_THIRD * 2], [MID_X,MIN_X]]

Enter the team trigraph and the date of the game to pull the data from MLB and chart it.

In [ ]:
# Enter team trigraph and date of game
# ====================================
team = 'WSH'
game_date = '10/30/2019'
# ====================================

The following cell contains utility functions used later to retrieve data and build plots.

In [ ]:
# get data from MLB
def get_data(url):
    try:
        results = requests.get(url).json()
        if 'messageNumber' in results:
            print('ERROR:  {} - {}'.format(results['messageNumber'], results['message']))
    except:
        print('An unhandled exception occurred retrieving data from MLB.\n')
    return results

# load the MLB team names into a dictionary (tri_graph: team_name)
def load_teams():
    fields = 'teams,0,id,name,abbreviation'
    teams = get_data(API_TEAMS_URL.format(fields))['teams']

    # load dictionaries using specific attributes of teams
    for team in teams:
        TEAM_NAMES_BY_ABBREV.update({team['abbreviation']: team['name']})

    return

# find the game key based upon the team playing and the date.  The game key is required
# for all MLB API calls.
def find_gamepk(team, game_date):
    game_pk = 0  # no game today
    away_team = ''
    home_team = ''
    
    try:
        # validate team names
        if team in TEAM_NAMES_BY_ABBREV.keys():
            fields = 'dates,games,gamePk,teams,team,name,gameNumber'
            schedule = get_data(API_SCHEDULE_URL.format(game_date, fields))

            # loop through games looking for team
            for games in schedule['dates'][0]['games']:

                if games['teams']['away']['team']['name'] == TEAM_NAMES_BY_ABBREV[team]: 
                    game_pk = games['gamePk']
                    away_team = games['teams']['away']['team']['name']
                    home_team = games['teams']['home']['team']['name']
                    break
                elif games['teams']['home']['team']['name'] == TEAM_NAMES_BY_ABBREV[team]:
                    game_pk = games['gamePk']
                    away_team = games['teams']['away']['team']['name']
                    home_team = games['teams']['home']['team']['name']   
                    break
    except:
        game_pk = 0  # no game found
    return game_pk, away_team, home_team

The following cell retrieves the batting data for a game using the MLB API.  Only the data required for building the spray charts are retrieved (specified in the _field_ variable).  Data is returned for hits and outs.

In [ ]:
def get_hit_data(game_pk):
    
    fields = "allPlays,result,event,about,halfInning,playEvents,hitData,location,coordinates,totalDistance"
    data = get_data(API_PLAYBYPLAY_URL.format(game_pk, fields))

    # Process batting data
    hit_data_tuple = []
    out_data_tuple = []
    for n in range(len(data['allPlays'])):
        #print('n=',n)
        play = data['allPlays'][n]
        #print('play=',play)
        inning_half = play['about']['halfInning']
        #print(inning_half)
        
        hit_result = play['result']['event']
        #print('hit result',hit_result)
        
        m = len(play['playEvents']) -1
        #print('m=',m)
        event = play['playEvents'][m]
        #print('event=',event)

        if 'hitData' in event.keys():
            location = int(event['hitData']['location'])
            x = event['hitData']['coordinates']['coordX']
            y = event['hitData']['coordinates']['coordY']
            
            if hit_result in ['Single', 'Double', 'Triple', 'Home Run']:
                hit_data_tuple.append((inning_half.upper(), x, y, location, hit_result))
            else:
                out_data_tuple.append((inning_half.upper(), x, y, location, hit_result))
                
    #hit_data = pd.DataFrame(data=hit_data_tuple, columns=['inning_half', 'x', 'y', 'location', 'play_result'])
    #print(hit_data.head(30))
    #out_data = pd.DataFrame(data=out_data_tuple, columns=['inning_half', 'x', 'y', 'location', 'play_result'])
    #print(out_data.head(30))
    return hit_data_tuple, out_data_tuple

The following function draws the baseball field and the three pie wedges.  These geometric figures are based on the chart constants defined in the first cell, as well as some aesthetic tweeking.

In [ ]:
def draw_field(team):
    
    # arcs can only be applied through patches, which require axes, thus the subplot()
    fig, ax = plt.subplots(1,1)
    fig.suptitle('Batting results for {} - {}'.format(team, game_date), fontsize=20)
    ax.set_xlim(0, MAX_X)
    ax.set_ylim(0, MAX_Y)
    ax.set_xticks(())
    ax.set_yticks(())
    
    # draw outfield arc - coordinates and angles tweeked for aesthetics
    ax.add_patch(mpatches.Arc([MID_X,205], MAX_X + 36, MAX_Y, angle=0, theta1=35, theta2=151))
    
    # draw infield arc - coordinates and angles tweeked for aesthetics
    ax.add_patch(mpatches.Arc([MID_X,MIN_Y], MAX_X/2, MAX_Y - 30, angle=0, theta1=70, theta2=110))
      
    # draw sector bounding lines
    px = [x[0] for x in POLY_LEFT_PTS]
    py = [y[1] for y in POLY_LEFT_PTS]
    ax.plot(px, py, color='black', linestyle='-', linewidth=1)
    
    px = [x[0] for x in POLY_MIDDLE_PTS]
    py = [y[1] for y in POLY_MIDDLE_PTS]
    ax.plot(px, py, color='black', linestyle='-', linewidth=1)
    
    px = [x[0] for x in POLY_RIGHT_PTS]
    py = [y[1] for y in POLY_RIGHT_PTS]
    ax.plot(px, py, color='black', linestyle='-', linewidth=1)
    
    return ax

This function plots the hits and outs for the given team.

In [ ]:
def plot_hits(inning_half, team, hit_data_tuple, out_data_tuple):
    
    # draw field boundaries and sectors
    ax = draw_field(team)
    
    # build polygons of sectors to check each hit coords
    poly_left = path.Path(POLY_LEFT_PTS)
    poly_middle = path.Path(POLY_MIDDLE_PTS)
    poly_right = path.Path(POLY_RIGHT_PTS)
    
    # put hit coordinates in list based upon inclusion in sector polys
    left_hit = [[x[1], Y_OFFSET-x[2], x[4]] for x in hit_data_tuple if poly_left.contains_point([x[1],Y_OFFSET-x[2]]) \
                and inning_half == x[0]]
    middle_hit = [[x[1], Y_OFFSET-x[2], x[4]] for x in hit_data_tuple if poly_middle.contains_point([x[1],Y_OFFSET-x[2]]) \
                  and inning_half == x[0]]
    right_hit = [[x[1], Y_OFFSET-x[2], x[4]] for x in hit_data_tuple if poly_right.contains_point([x[1],Y_OFFSET-x[2]]) \
                 and inning_half == x[0]]
        
    # seperate points into respective lists of x and y coordinates
    left_x = [x[0] for x in left_hit]
    left_y = [y[1] for y in left_hit]
    right_x = [x[0] for x in right_hit]
    right_y = [y[1] for y in right_hit]
    mid_x = [x[0] for x in middle_hit]
    mid_y = [y[1] for y in middle_hit]

    # plot hits
    ax.scatter(left_x + mid_x + right_x, left_y + mid_y + right_y, color='blue', label='Hit')
    
    # put out coordinates in list based upon inclusion in sector polys
    left_out = [[x[1], 400-x[2], x[4]] for x in out_data_tuple if poly_left.contains_point([x[1],400-x[2]]) \
                and inning_half == x[0]]
    middle_out = [[x[1], 400-x[2], x[4]] for x in out_data_tuple if poly_middle.contains_point([x[1],400-x[2]]) \
                  and inning_half == x[0]]
    right_out = [[x[1], 400-x[2], x[4]] for x in out_data_tuple if poly_right.contains_point([x[1],400-x[2]]) \
                 and inning_half == x[0]]
    
    # seperate points into respective lists of x and y coordinates
    left_x = [x[0] for x in left_out]
    left_y = [y[1] for y in left_out]
    right_x = [x[0] for x in right_out]
    right_y = [y[1] for y in right_out]
    mid_x = [x[0] for x in middle_out]
    mid_y = [y[1] for y in middle_out]

    # plot outs    
    ax.scatter(left_x + mid_x + right_x, left_y + mid_y + right_y, color='red', label='Out', marker='x')
    
    ax.legend()
    plt.show();
    

The is the main body of the code.  This code loads the team name dictionary, finds the game key and formal team names, retrieves the batting data from MLB, and plots the hit results for the away and home team.

In [ ]:
# ===== MAIN =====

# load team names and abbrev
load_teams()

# Get game PK
game_pk, away_team, home_team = find_gamepk(team, game_date)

# Retrieve hit data
if game_pk > 0:
    hit_data, out_data = get_hit_data(game_pk)
    
    # Plot the pitch data
    plot_hits('TOP', away_team, hit_data, out_data)
    plot_hits('BOTTOM', home_team, hit_data, out_data)
    
else:
    print("No game found for {} on {}".format(team, game_date))
    print(TEAM_NAMES_BY_ABBREV)